<a href="https://colab.research.google.com/github/cvphelps/ml-examples/blob/master/PyTorch_CNN_MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Welcome!
In this tutorial we'll walk through a simple convolutional neural network to classify the handwritten digits in MNIST using PyTorch.

We’ll also see how we can use Weights and Biases to log our model's metrics, inspect its performance and share our findings about the best architecture for our neural network!

Try forking this notebook and tweaking some hyperparameters to hone your intuition.

All the results will be going into the [shared W&B project page](https://app.wandb.ai/wandb/pytorch-mnist).

![alt text](https://i.imgur.com/8SPcpvn.png)

## Setup

Here we add a few lines of code to:


*   **pip install wandb** – Install the W&B library
*   **import wandb** – Import the wandb library
*   **wandb login** – Login to your W&B account so you can log all your metrics in one place
*   **wandb.init()** – Initialize a new W&B run



In [0]:
# WandB – Install the W&B library
!pip install wandb -q

In [0]:
from __future__ import print_function
import argparse
import random # to set the python random seed
import numpy # to set the numpy random seed
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
# Ignore excessive warnings
import logging
logging.propagate = False 
logging.getLogger().setLevel(logging.ERROR)

# WandB – Import the wandb library
import wandb

The script will prompt you to create a free W&B account to track your model metrics and save your progress.

In [0]:
# WandB – Login to your wandb account so you can log all your metrics
!wandb login

# Defining a Model

## Create Your Neural Network

In [0]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        # In our constructor, we define our neural network architecture that we'll use in the forward pass.
        # Conv2d() adds a convolution layer that generates 2 dimensional feature maps to learn different aspects of our image
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        
        # Dropout randomly turns off a percentage of neurons at each training step resulting
        # in a more robust neural network that is resistant to overfitting
        self.conv2_drop = nn.Dropout2d()
        
        # Linear(x,y) creates dense, fully connected layers with x inputs and y outputs
        # Linear layers simply output the dot product of our inputs and weights.
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        # Here we feed the feature maps from the convolutional layers into a max_pool2d layer.
        # The max_pool2d layer reduces the size of the image representation our convolutional layers learnt,
        # and in doing so it reduces the number of parameters and computations the network needs to perform.
        # Finally we apply the relu activation function which gives us max(0, max_pool2d_output)
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        
        # Reshapes x into size (-1, 320) so we can feed the convolution layer outputs into our fully connected layer
        x = x.view(-1, 320)
        
        # We apply the relu activation function and dropout to the output of our fully connected layers
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        
        # Finally we apply the softmax function to squash the probabilities of each class (0-9) and ensure they add to 1.
        return F.log_softmax(x, dim=1)

## Train Your Neural Network

In [0]:
def train(args, model, device, train_loader, optimizer, epoch):
    # Switch model to training mode. This is necessary for layers like dropout, batchnorm etc which behave differently in training and evaluation mode
    model.train()
    
    # We loop over the data iterator, and feed the inputs to the network and adjust the weights.
    for batch_idx, (data, target) in enumerate(train_loader):
        if batch_idx > 20:
          break
        # Load the input features and labels from the training dataset
        data, target = data.to(device), target.to(device)
        
        # Reset the gradients to 0 for all learnable weight parameters
        optimizer.zero_grad()
        
        # Forward pass: Pass image data from training dataset, make predictions about class image belongs to (0-9 in this case)
        output = model(data)
        
        # Define our loss function, and compute the loss
        loss = F.nll_loss(output, target)
        
        # Backward pass: compute the gradients of the loss w.r.t. the model's parameters
        loss.backward()
        
        # Update the neural network weights
        optimizer.step()

## Evaluate Your Neural Network

Here we add a line of code to:

*   **wandb.log()** – Log your metrics (accuracy, loss and epoch) and examples of images along with the predicted and true labels. This allows you to visualize your neural network's performance over time.
*   **wandb.watch()** – Fetch all layer dimensions, gradients, model parameters and log them automatically to your dashboard.
*   **wandb.save()** – Save the model checkpoint.

In [0]:
def test(args, model, device, test_loader):
    # Switch model to evaluation mode. This is necessary for layers like dropout, batchnorm etc which behave differently in training and evaluation mode
    model.eval()
    test_loss = 0
    correct = 0

    example_images = []
    with torch.no_grad():
        for data, target in test_loader:
            # Load the input features and labels from the test dataset
            data, target = data.to(device), target.to(device)
            
            # Make predictions: Pass image data from test dataset, make predictions about class image belongs to (0-9 in this case)
            output = model(data)
            
            # Compute the loss sum up batch loss
            test_loss += F.nll_loss(output, target, reduction='sum').item()
            
            # Get the index of the max log-probability
            pred = output.max(1, keepdim=True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()
            
            # WandB – Log images in your test dataset automatically, along with predicted and true labels by passing pytorch tensors with image data into wandb.Image
            example_images.append(wandb.Image(
                data[0], caption="Pred: {} Truth: {}".format(pred[0].item(), target[0])))
    
    # WandB – wandb.log(a_dict) logs the keys and values of the dictionary passed in and associates the values with a step.
    # You can log anything by passing it to wandb.log, including histograms, custom matplotlib objects, images, video, text, tables, html, pointclouds and other 3D objects.
    # Here we use it to log test accuracy, loss and some test images (along with their true and predicted labels).
    wandb.log({
        "Examples": example_images,
        "Test Accuracy": 100. * correct / len(test_loader.dataset),
        "Test Loss": test_loss})

# Train, Edit, and Retrain
Run wandb.init() each time you start a new run. For this tutorial we're logging results to an open, shared project called "[pytorch-mnist](https://app.wandb.ai/wandb/pytorch-mnist/)" so everyone's results go into the same project. (The entity refers to the team name this project is under.)

You can see your classmates' runs on the [project page](https://app.wandb.ai/wandb/pytorch-mnist/).

Edit the config values to change hyperparameters. Then re-run these two cells below to train a new version of the model.

## Initialize Hyperparameters

Here we add a few lines of code to:


*   **wandb.config** – Save all your hyperparameters in a config object. This lets you sort and search through your runs by hyperparameter values.

We encourage you to tweak these and run this notebook again to see if you can achieve improved model performance!

In [20]:
# WandB – Initialize a new run
wandb.init(entity="wandb", project="pytorch-mnist")
wandb.watch_called = False # Re-run the model without restarting the runtime, unnecessary after our next release

# WandB – Config is a variable that holds and saves hyperparameters and inputs
config = wandb.config          # Initialize config
config.batch_size = 32          # input batch size for training (default: 64)
config.test_batch_size = 24    # input batch size for testing (default: 1000)
config.epochs = 60             # number of epochs to train (default: 10)
config.lr = 0.1               # learning rate (default: 0.01)
config.momentum = 0.1          # SGD momentum (default: 0.5) 
config.no_cuda = False         # disables CUDA training
config.seed = 42               # random seed (default: 42)
config.log_interval = 10     # how many batches to wait before logging training status

In [0]:
%%wandb 
## WandB — show a preview of live graphs here in the notebook

def main():
    use_cuda = not config.no_cuda and torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")
    kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
    
    # Set random seeds and deterministic pytorch for reproducibility
    random.seed(config.seed)       # python random seed
    torch.manual_seed(config.seed) # pytorch random seed
    numpy.random.seed(config.seed) # numpy random seed
    torch.backends.cudnn.deterministic = True

    # Load the dataset: We're training our CNN on MNIST which consists of black and white images of hand-written digits, 0 to 9.
    train_loader = torch.utils.data.DataLoader(
        datasets.MNIST('../data', train=True, download=True,
                       transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=config.batch_size, shuffle=True, **kwargs)
    test_loader = torch.utils.data.DataLoader(
        datasets.MNIST('../data', train=False, transform=transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.1307,), (0.3081,))
        ])),
        batch_size=config.test_batch_size, shuffle=True, **kwargs)

    # Initialize our model, recursively go over all modules and convert their parameters and buffers to CUDA tensors (if device is set to cuda)
    model = Net().to(device)
    optimizer = optim.SGD(model.parameters(), lr=config.lr,
                          momentum=config.momentum)
    
    # WandB – wandb.watch() automatically fetches all layer dimensions, gradients, model parameters and logs them automatically to your dashboard.
    # Using log="all" log histograms of parameter values in addition to gradients
    wandb.watch(model, log="all")

    for epoch in range(1, config.epochs + 1):
        train(config, model, device, train_loader, optimizer, epoch)
        test(config, model, device, test_loader)
        
    # WandB – Save the model checkpoint. This automatically saves a file to the cloud and associates it with the current run.
    torch.save(model.state_dict(), "model.h5")
    wandb.save('model.h5')

if __name__ == '__main__':
    main()

Check out the [project page](https://app.wandb.ai/wandb/pytorch-mnist/) to see your results coming in to the shared project.

Take a look at the graphs tracking loss and accuracy across different experiments trying to develop a more accurate model. Press 'option+space' to expand the runs table, comparing all the results from everyone who has tried this script. Next, See if you can tweak the hyperparameters in the "Initialize Hyperparameters" section. Your goal is to maximize Test Accuracy. Good luck!

# More about W&B
We're always free for academics and open source projects. Email carey@wandb.com with any questions or feature suggestions.



**Resources**
1. [Documentation](http://docs.wandb.com) - Python docs
2. [Gallery](https://app.wandb.ai/gallery) - example reports in W&B